In [ ]:
!pip install --upgrade --force-reinstall --no-deps kaggle #installing kaggle

     |████████████████████████████████| 61kB 2.7MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.8-cp36-none-any.whl size=73275 sha256=385a244f9602b7015fb144462dc6d5767bc35e8a254433fbd6f6a866507086db
  Stored in directory: /root/.cache/pip/wheels/94/a7/09/68dc83c7c14fdbdf5d3f2b2da5b87e587bfc1e85df69b1130c
Successfully built kaggle
  Found existing installation: kaggle 1.5.8
    Uninstalling kaggle-1.5.8:
      Successfully uninstalled kaggle-1.5.8


In [ ]:
from google.colab import drive #importing drive
drive.mount('/content/drive/') #mounting drive

Mounted at /content/drive/


In [ ]:
!mkdir -p ~/.kaggle 
!cp '/content/drive/My Drive/kaggle.json' ~/.kaggle/

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c plant-seedlings-classification #to download the dataset of plant seedling classfication

 99% 1.68G/1.69G [00:25<00:00, 51.3MB/s]
100% 1.69G/1.69G [00:25<00:00, 72.0MB/s]


In [ ]:
!cp -r 'plant-seedlings-classification.zip' 'drive/My Drive/plant-seedlings-classification.zip'

In [ ]:
!unzip plant-seedlings-classification.zip #unziping the dataset

Streaming output truncated to the last 5000 lines.
  inflating: test/adb7a032c.png      
  inflating: test/ae90f2827.png      
  inflating: test/aecfaed64.png      
  inflating: test/aee6fa3df.png      
  inflating: test/af45e222a.png      
  inflating: test/afa446484.png      
  inflating: test/afcf6abd5.png      
  inflating: test/b026bf8ca.png      
  inflating: test/b03397525.png      
  inflating: test/b0acaff4a.png      
  inflating: test/b130a0632.png      
  inflating: test/b145ba9d4.png      
  inflating: test/b1cd2a91e.png      
  inflating: test/b215531dd.png      
  inflating: test/b2706e2b3.png      
  inflating: test/b29339405.png      
  inflating: test/b30ab4659.png      
  inflating: test/b31292706.png      
  inflating: test/b341d0aab.png      
  inflating: test/b39c71707.png      
  inflating: test/b3d6fdb80.png      
  inflating: test/b3e08b037.png      
  inflating: test/b47691c08.png      
  inflating: test/b4c3df835.png      
  inflating: test/b4f7c9214.png      

In [ ]:
# importing libraries
from IPython.display import clear_output 
import numpy as np
import cv2
import pandas as pd
import tensorflow as tf
import os
import matplotlib.pyplot as plt

In [ ]:
catg= os.listdir("train")
catg

['Maize',
 'Cleavers',
 'Small-flowered Cranesbill',
 'Scentless Mayweed',
 'Shepherds Purse',
 'Common wheat',
 'Black-grass',
 'Fat Hen',
 'Common Chickweed',
 'Sugar beet',
 'Loose Silky-bent',
 'Charlock']

In [ ]:
#dataset preprocessing
x=[]  #list to store processed plant images
y=[]  #list to store category of the plant images
S1,S2 = 140,140       
def data():
    count=0
    path = "/content/"
    ty = ["train"]
    for typ in ty:
        dir = os.path.join(path,typ)
        for ele in catg:
            pa = os.path.join(dir,ele)
            class_num = catg.index(ele)
            for img in os.listdir(pa):
                image = os.path.join(pa, img)
                image = cv2.imread(image, cv2.IMREAD_ANYCOLOR)
                image = cv2.resize(image , (S1, S2))
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                x.append(image)
                x.append(cv2.flip(image,-1))
                y.append(class_num)
                y.append(class_num)
                count+=1
        clear_output()
        print(count)

In [ ]:
data() #calling function for preprocessing

4750


In [ ]:
def one_hot(labels,C):
    One_hot_matrix = tf.one_hot(labels,C)
    return tf.keras.backend.eval(One_hot_matrix)
y = one_hot(y, 12)
print ("Y shape: " + str(y.shape))

Y shape: (9500, 12)


In [ ]:
x = np.array(x)/255. #normalising the dataset
y = np.array(y)
x.shape

(9500, 140, 140, 3)

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.20, random_state = 8)   #split the data in training and testing sets

In [ ]:
del x
del y

In [ ]:
#defining the cnn model
model1 = tf.keras.Sequential([
    tf.keras.layers.Conv2D(64, 1, activation=None,kernel_regularizer=tf.keras.regularizers.l2(0.1), input_shape=(140,140,3)),
    tf.keras.layers.BatchNormalization(axis=3),
    tf.keras.layers.LeakyReLU(0.1),
    tf.keras.layers.MaxPool2D(strides=2),
    tf.keras.layers.Conv2D(128, 3, activation=None,padding="same",kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.BatchNormalization(axis=3),
    tf.keras.layers.LeakyReLU(0.1),
    tf.keras.layers.MaxPool2D(strides=2),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Conv2D(256, 5, activation=None,kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.BatchNormalization(axis=3),
    tf.keras.layers.LeakyReLU(0.1),
    tf.keras.layers.MaxPool2D(strides=2),   
    tf.keras.layers.Conv2D(64, 5, activation=None,padding="same",kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.BatchNormalization(axis=3),
    tf.keras.layers.LeakyReLU(0.1),
    tf.keras.layers.MaxPool2D(strides=2),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(100,kernel_regularizer=tf.keras.regularizers.l2(0.01), activation=None),
    tf.keras.layers.BatchNormalization(axis=1),
    tf.keras.layers.ReLU(),
     tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(50,kernel_regularizer=tf.keras.regularizers.l2(0.01), activation=None),
    tf.keras.layers.BatchNormalization(axis=1),
    tf.keras.layers.ReLU(),
    tf.keras.layers.Dense(12, kernel_regularizer=tf.keras.regularizers.l2(0.01) ,activation='softmax')
])

In [ ]:
ilr = 0.001 #initial rate
# Rate decay with exponential decay
lr = tf.keras.optimizers.schedules.ExponentialDecay(ilr,decay_steps=800,decay_rate=0.5,staircase=True)

In [ ]:
#model compile
model1.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr),loss=tf.keras.losses.CategoricalCrossentropy(),metrics=['accuracy'])

In [ ]:
result = model1.fit(x=x_train,y=y_train,batch_size=64,epochs=60,verbose=1,shuffle=False,initial_epoch=0,
                   validation_split=0.2)

Epoch 1/60
95/95 [==============================] - 9s 91ms/step - loss: 6.8949 - accuracy: 0.3717 - val_loss: 6.2775 - val_accuracy: 0.1349
Epoch 2/60
95/95 [==============================] - 8s 88ms/step - loss: 3.1361 - accuracy: 0.6286 - val_loss: 4.5631 - val_accuracy: 0.1349
Epoch 3/60
95/95 [==============================] - 8s 87ms/step - loss: 2.0834 - accuracy: 0.7012 - val_loss: 3.9523 - val_accuracy: 0.1349
Epoch 4/60
95/95 [==============================] - 8s 87ms/step - loss: 1.7142 - accuracy: 0.7444 - val_loss: 4.1853 - val_accuracy: 0.1349
Epoch 5/60
95/95 [==============================] - 8s 87ms/step - loss: 1.5921 - accuracy: 0.7551 - val_loss: 4.5503 - val_accuracy: 0.1349
Epoch 6/60
95/95 [==============================] - 8s 86ms/step - loss: 1.4799 - accuracy: 0.7666 - val_loss: 5.2681 - val_accuracy: 0.0520
Epoch 7/60
95/95 [==============================] - 8s 86ms/step - loss: 1.4517 - accuracy: 0.7735 - val_loss: 4.1465 - val_accuracy: 0.1112
Epoch 8/60
95

In [ ]:
result.history.keys()

dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy'])

In [ ]:
va = model1.evaluate(x_test,y_test,verbose=2) #model evaluate on test set (train test split)

60/60 - 1s - loss: 0.3442 - accuracy: 0.9600


In [ ]:
rec = []

In [ ]:
rec.append(va)

In [ ]:
rec

[[0.3442075252532959, 0.9599999785423279]]

In [ ]:
#preprocessing of test images
x=[]
file = []
def createData():
    count=0
    path = "/content/"
    typee = ["test"]
    for typ in typee:
        Pa = os.path.join(path,typ)
        for img in os.listdir(Pa):
            file.append(img)
            image = os.path.join(Pa, img)
            image = cv2.imread(image, cv2.IMREAD_ANYCOLOR)
            image = cv2.resize(image , (S1, S2))
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            x.append(image)
            count+=1
    print(count)

In [ ]:
createData()
x = np.array(x)/255. #normalize the test images 
x.shape

794


(794, 140, 140, 3)

In [ ]:
#predicting the species of the test images
spec = model1.predict_classes(x)
print(spec.shape)
print(spec)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
(794,)
[ 8  7  9  7  4  6  8 11  0  9  5 11 10  0  9 10  3 10  8  4  7  4  3  1
 10  6  7  8  0  4 11 11  8  9  9  8  7  8  9  7  4 11  3 10 10 10  5  3
  7 10  7  1 11 10  7  1 10  8  9 10  9  2 11  9 11  7  2 10  2  2  1 10
 11  9 10  3 10  2  3  9  9  3  1 11  7  4 10  4  8  7  6 11  0  3  3  2
  8  1  6  8  7 11  2  2  7  8  8  7  0  3  2  9  8  3  8  2  3 10 11 11
  3  7  7 11  2  4  6  6  5  3  7  8 11  6 10  2  2  4 10  3  9  2  0  2
  3 11  2  4  9  7  7  9  8  5  8  0  3  0  3  8  4  6 11  2  8 11  2 11
  3  4 10  1  9 11  6 11  9 10  8  9  4  4  8  7  2 10 10  2  3  8 10  3
  1  8  2  8 10  7  8  5  2  9  8  2 10  8  3 11  7  8  9  0 10  4 11  6
  

In [ ]:
res = pd.DataFrame({'file':file, 'species':spec})

In [ ]:
res["species"] = res["species"].apply(lambda x: catg[int(x)])
res.head(20)

,file,species
0,0086a6340.png,Common Chickweed
1,33317fc2a.png,Fat Hen
2,e1abb4ff9.png,Sugar beet
3,d3331e071.png,Fat Hen
4,099b961ec.png,Shepherds Purse
5,0bf7bfb05.png,Black-grass
6,92292055d.png,Common Chickweed
7,cfd8165e9.png,Charlock
8,534e74d83.png,Maize
9,0dba99002.png,Sugar beet


In [ ]:
res.to_csv("answers.csv",index=False)

In [ ]:
model1.save("saved_model1")

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: saved_model1/assets


In [ ]:
!kaggle competitions submit -c plant-seedlings-classification -f answers.csv -m "first"

100% 22.2k/22.2k [00:03<00:00, 7.32kB/s]
Successfully submitted to Plant Seedlings Classification